# Project Computer Vision - Task 1

### Libraries

In [1]:
import cv2
import os
import numpy as np
import json

### Data directory

In [2]:
imagesDir = './samples'
testDir = './json_examples/'

### Leading with input file

In [3]:
# Build the path to the input.json file
json_file_path = os.path.join(testDir, 'input.json')

# Open and read the input.json
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Extract image paths into a list
image_paths = data['image_files']

# Print the list of image paths
print(image_paths)

['IMG_20201127_235047.jpg', 'IMG_20201127_234711.jpg', '2_7mUR_original-test_1609981034838.jpg']


In [10]:
# Opening an image
first_image = image_paths[0]
image_to_test = cv2.imread(os.path.join(imagesDir, first_image))
image_to_test = cv2.resize(image_to_test, (0, 0), fx = 0.15, fy = 0.15)

# Showing the image
cv2.imshow("Lego", image_to_test)

# Waiting for user to press a key to close the image
cv2.waitKey(0)

# Close the window after user pressed a key
cv2.destroyWindow("Lego")

## Filtering and Smoothing

#### MEAN FILTER

In [11]:
# Apply mean filter to the image
img_mean_filter = cv2.blur(image_to_test, (4,4))

# Show image
cv2.imshow('mean_filter', img_mean_filter)
cv2.waitKey(0)
cv2.destroyWindow('mean_filter')

#### GAUSSIAN FILTER

In [12]:
# Apply gaussian filter to the image
img_gaussian_filter = cv2.GaussianBlur(image_to_test, (5,5), 0)

# Show image
cv2.imshow('gaussian_filter', img_gaussian_filter)
cv2.waitKey(0)
cv2.destroyWindow('gaussian_filter')

#### MEDIAN FILTER

In [13]:
# Apply median filter to the image
img_median_filter = cv2.medianBlur(image_to_test, 5)

# Show image
cv2.imshow('median_filter', img_median_filter)
cv2.waitKey(0)
cv2.destroyWindow('median_filter')

#### BILATERAL FILTER

In [14]:
# Apply bilateral filter to the image
img_bilateral_filter = cv2.bilateralFilter(image_to_test, 9, 75, 75)

# Show image
cv2.imshow('bilateral_filter', img_bilateral_filter)
cv2.waitKey(0)
cv2.destroyWindow('bilateral_filter')

## Edge and Line Detection


In [15]:
# Convert the image to grayscale
gray_img = cv2.cvtColor(image_to_test, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray_img, 100, 200)

cv2.imshow('canny_edge_detection', edges)
cv2.waitKey(0)
cv2.destroyWindow('canny_edge_detection')